## Exploração e Manipulação de um dataset de carros da Toyota com Pyspark

In [1]:
import os
os.environ['SPARK_HOME'] = "C:/Users/jonat/Jupyter/spark-3.2.1-bin-hadoop2.7"

In [2]:
import findspark
findspark.init()

### Criando uma spark session

In [3]:
from pyspark.sql import SparkSession
spark = SparkSession.builder\
        .master('local')\
        .appName('sparkjupyter')\
        .getOrCreate()

In [4]:
#importar o dataset
tabela = spark.read.csv('C:/Users/jonat/Jupyter/Toyota.csv', header=True, inferSchema=True)

In [5]:
tabela.show()

+---+-----+---+-----+--------+----+--------+---------+----+-----+------+
|_c0|Price|Age|   KM|FuelType|  HP|MetColor|Automatic|  CC|Doors|Weight|
+---+-----+---+-----+--------+----+--------+---------+----+-----+------+
|  0|13500| 23|46986|  Diesel|  90|       1|        0|2000|three|  1165|
|  1|13750| 23|72937|  Diesel|  90|       1|        0|2000|    3|  1165|
|  2|13950| 24|41711|  Diesel|  90|    null|        0|2000|    3|  1165|
|  3|14950| 26|48000|  Diesel|  90|       0|        0|2000|    3|  1165|
|  4|13750| 30|38500|  Diesel|  90|       0|        0|2000|    3|  1170|
|  5|12950| 32|61000|  Diesel|  90|       0|        0|2000|    3|  1170|
|  6|16900| 27|   ??|  Diesel|????|    null|        0|2000|    3|  1245|
|  7|18600| 30|75889|    null|  90|       1|        0|2000|    3|  1245|
|  8|21500| 27|19700|  Petrol| 192|       0|        0|1800|    3|  1185|
|  9|12950| 23|71138|  Diesel|????|    null|        0|1900|    3|  1105|
| 10|20950| 25|31461|  Petrol| 192|       0|       

In [6]:
print((tabela.count(), len(tabela.columns)))

(1436, 11)


In [7]:
tabela.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- KM: string (nullable = true)
 |-- FuelType: string (nullable = true)
 |-- HP: string (nullable = true)
 |-- MetColor: integer (nullable = true)
 |-- Automatic: integer (nullable = true)
 |-- CC: integer (nullable = true)
 |-- Doors: string (nullable = true)
 |-- Weight: integer (nullable = true)



## Manipulando os dados

#### Tratando a coluna 'Doors'

In [8]:
tabela.groupBy('Doors').count().orderBy('count', ascending=False).show()

+-----+-----+
|Doors|count|
+-----+-----+
|    5|  673|
|    3|  621|
|    4|  137|
|    2|    2|
| four|    1|
|three|    1|
| five|    1|
+-----+-----+



In [9]:
tabela = tabela.na.replace({'four': '4','three':'3','five':'5'})
tabela.groupBy('Doors').count().orderBy('count', ascending=False).show()

+-----+-----+
|Doors|count|
+-----+-----+
|    5|  674|
|    3|  622|
|    4|  138|
|    2|    2|
+-----+-----+



In [10]:
#Apagando a coluna Doors e criando uma nova coluna doors do tipo int
tabela = tabela.withColumn("Doors",tabela["Doors"].cast('integer'))
tabela.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Price: integer (nullable = true)
 |-- Age: integer (nullable = true)
 |-- KM: string (nullable = true)
 |-- FuelType: string (nullable = true)
 |-- HP: string (nullable = true)
 |-- MetColor: integer (nullable = true)
 |-- Automatic: integer (nullable = true)
 |-- CC: integer (nullable = true)
 |-- Doors: integer (nullable = true)
 |-- Weight: integer (nullable = true)



#### Localizando valores nulos ou NaN

In [11]:
from pyspark.sql.functions import col,isnan, when, count

In [12]:
tabela.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in tabela.columns]
   ).show()

+---+-----+---+---+--------+---+--------+---------+---+-----+------+
|_c0|Price|Age| KM|FuelType| HP|MetColor|Automatic| CC|Doors|Weight|
+---+-----+---+---+--------+---+--------+---------+---+-----+------+
|  0|    0|100|  0|     100|  0|     150|        0|  0|    0|     0|
+---+-----+---+---+--------+---+--------+---------+---+-----+------+



#### Tratando a coluna 'MetColor'
a coluna possui valores binários, os campos null serão substituidos por 0

In [13]:
tabela.groupBy('MetColor').count().orderBy('count', ascending=False).show()

+--------+-----+
|MetColor|count|
+--------+-----+
|       1|  868|
|       0|  418|
|    null|  150|
+--------+-----+



In [14]:
tabela = tabela.na.fill(value=0,subset=["MetColor"])
tabela.groupBy('MetColor').count().orderBy('count', ascending=False).show()

+--------+-----+
|MetColor|count|
+--------+-----+
|       1|  868|
|       0|  568|
+--------+-----+



#### Coluna FuelType

In [15]:
tabela = tabela.na.fill(value='nao informado',subset=["FuelType"])
tabela.groupBy('FuelType').count().orderBy('count', ascending=False).show()

+-------------+-----+
|     FuelType|count|
+-------------+-----+
|       Petrol| 1177|
|       Diesel|  144|
|nao informado|  100|
|          CNG|   15|
+-------------+-----+



#### Coluna Age
Os valores null da tabela 'Age' serão substituidos pelos valores mais frequentes

In [22]:
tabela.groupBy('Age').count().orderBy('count', ascending=False).show(5)

+---+-----+
|Age|count|
+---+-----+
| 62|  141|
| 65|   62|
| 68|   60|
| 80|   52|
| 78|   41|
+---+-----+
only showing top 5 rows



In [17]:
tabela = tabela.na.fill(value=62,subset=["Age"])

In [18]:
tabela.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in tabela.columns]
   ).show()

+---+-----+---+---+--------+---+--------+---------+---+-----+------+
|_c0|Price|Age| KM|FuelType| HP|MetColor|Automatic| CC|Doors|Weight|
+---+-----+---+---+--------+---+--------+---------+---+-----+------+
|  0|    0|  0|  0|       0|  0|       0|        0|  0|    0|     0|
+---+-----+---+---+--------+---+--------+---------+---+-----+------+



In [20]:
# Selecionando um subconjunto do dataset
select = tabela.select("Price", "Age", "FuelType")
select.show(5)

+-----+---+--------+
|Price|Age|FuelType|
+-----+---+--------+
|13500| 23|  Diesel|
|13750| 23|  Diesel|
|13950| 24|  Diesel|
|14950| 26|  Diesel|
|13750| 30|  Diesel|
+-----+---+--------+
only showing top 5 rows



In [21]:
# Filtrando dados
tabela.filter(tabela.Age < 20).show(5)

+---+-----+---+-----+--------+---+--------+---------+----+-----+------+
|_c0|Price|Age|   KM|FuelType| HP|MetColor|Automatic|  CC|Doors|Weight|
+---+-----+---+-----+--------+---+--------+---------+----+-----+------+
| 88|15950| 19|51884|  Petrol| 97|       1|        0|1400|    3|  1100|
| 89|21950| 19|50005|  Diesel|110|       1|        0|2000|    3|  1265|
| 95|19950| 17|30351|  Diesel| 90|       1|        0|1995|    3|  1260|
| 96|15950| 19|29435|  Petrol| 97|       1|        0|1400|    3|  1100|
| 97|15950| 19|25948|  Petrol| 97|       1|        0|1400|    3|  1100|
+---+-----+---+-----+--------+---+--------+---------+----+-----+------+
only showing top 5 rows



#### Agregando

In [23]:
#Encontrando o carro mais barato movido a Disel
tabela.filter(tabela.FuelType == "Diesel").groupBy().min("Price").show()

+----------+
|min(Price)|
+----------+
|      4350|
+----------+



In [31]:
#Encontrando o carro mais velho com cambio automático
tabela.filter(tabela.Automatic == 1).groupBy().max("Age").show()

+--------+
|max(Age)|
+--------+
|      80|
+--------+



In [33]:
# Encontrando o preço médio de um carro 4 portas movido a Diesel
tabela.filter(tabela.Doors == 4).filter(tabela.FuelType == "Diesel").groupBy().avg("Price").show()

+-----------------+
|       avg(Price)|
+-----------------+
|7912.583333333333|
+-----------------+



#### Visualizando desvio padrão da coluna Age

In [46]:
import pyspark.sql.functions as F
desv = tabela.groupBy("Price", "FuelType")
desv.agg(F.stddev("Age")).show(5)

+-----+--------+-----------------+
|Price|FuelType| stddev_samp(Age)|
+-----+--------+-----------------+
|15999|  Petrol|             null|
|24500|  Petrol|             null|
| 7750|     CNG|             null|
| 9650|  Petrol|9.899494936611665|
| 8500|  Diesel|4.349329450233296|
+-----+--------+-----------------+
only showing top 5 rows



### Fazendo uma consulta Sql (query)

In [54]:
#Registrando o dataframe em uma view temporária
tabela.createOrReplaceTempView("tabela")
# Selecionando as 10 primeiras linhas do dataset onde a idade é maior que 19
query = "FROM tabela SELECT * WHERE Age > 19 LIMIT 10"

In [55]:
consulta = spark.sql(query)
consulta.show()

+---+-----+---+-----+-------------+----+--------+---------+----+-----+------+
|_c0|Price|Age|   KM|     FuelType|  HP|MetColor|Automatic|  CC|Doors|Weight|
+---+-----+---+-----+-------------+----+--------+---------+----+-----+------+
|  0|13500| 23|46986|       Diesel|  90|       1|        0|2000|    3|  1165|
|  1|13750| 23|72937|       Diesel|  90|       1|        0|2000|    3|  1165|
|  2|13950| 24|41711|       Diesel|  90|       0|        0|2000|    3|  1165|
|  3|14950| 26|48000|       Diesel|  90|       0|        0|2000|    3|  1165|
|  4|13750| 30|38500|       Diesel|  90|       0|        0|2000|    3|  1170|
|  5|12950| 32|61000|       Diesel|  90|       0|        0|2000|    3|  1170|
|  6|16900| 27|   ??|       Diesel|????|       0|        0|2000|    3|  1245|
|  7|18600| 30|75889|nao informado|  90|       1|        0|2000|    3|  1245|
|  8|21500| 27|19700|       Petrol| 192|       0|        0|1800|    3|  1185|
|  9|12950| 23|71138|       Diesel|????|       0|        0|1900|